In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

ginf_df = pd.read_csv('/kaggle/input/football-events/ginf.csv')
ginf_df.head()

In [ ]:
# remove games with no events data
ginf_df = ginf_df[ginf_df['adv_stats']]
ginf_df = ginf_df[['id_odsp', 'fthg', 'ftag', 'odd_h', 'odd_a', 'odd_d']]
# Net home goals for regression target
ginf_df['net_h'] = ginf_df['fthg'] - ginf_df['ftag']
ginf_df = ginf_df.drop(columns=['fthg', 'ftag'])
# Change highest odds into multiclass categorical
cols = ['odd_h', 'odd_a', 'odd_d']
ginf_df["odds"] = ginf_df[cols].values.argmax(axis=1)
ginf_df = ginf_df.drop(columns=cols)
# Use net home goals for binary classification target
ginf_df['home_won'] = (ginf_df['net_h'] > 0).astype(int)

ginf_df = ginf_df.set_index('id_odsp')
print(ginf_df.info())
ginf_df.head(10)

In [ ]:
events_df = pd.read_csv('/kaggle/input/football-events/events.csv')
print(events_df.info())
events_df.head()

In [ ]:
events_df = events_df[['id_odsp', 'time', 'event_type', 'side']] # Some other interesting info but we're just going to combine these two columns for events
events_df['event'] = events_df['side'].map(str) + '-' + events_df['event_type'].map(str)
events_df = events_df.drop(columns=['event_type', 'side'])
# Sort by time
events_df = events_df.sort_values(by=['id_odsp', 'time'])
# Create TS column (last of time column)
target = events_df.groupby('id_odsp').tail(1).set_index('id_odsp').rename({'time': 'TS'}, axis=1).drop(columns=['event'])
print(target.head())
# Combine event rows into single column
events_df = events_df.groupby('id_odsp').agg({'time': lambda x: "*".join([str(y) for y in x]),
                                               'event': lambda x: "*".join([str(y) for y in x])})
print(events_df.head())
events_df = events_df.join(target)
events_df.head()

In [ ]:
blend_df = events_df.join(ginf_df)
print(blend_df.info())
blend_df.head()

In [ ]:
from IPython.display import FileLink
df_name = 'soccer_blend_df.parquet'
blend_df.to_parquet(df_name)
FileLink(df_name)